In [1]:
from openai import OpenAI

client = OpenAI()
text = {
        "format": {
            "type": "json_schema",
            "name": "repsponse_json_schema",
            "schema": {
                "type": "object",
                "properties": {
                    "statements": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "explanation": { "type": "string" },
                                "output": { "type": "string" }
                            },
                            "required": ["explanation", "output"],
                            "additionalProperties": False
                        }
                    }
                },
                "required": ["statements"],
                "additionalProperties": False
            },
            "strict": True
        }
    }

In [9]:
text = {
        "format": {
            "type": "json_schema",
            "name": "response_json_schema",
            "schema": {
                "type": "object",
                "properties": {
                            "explanation": { "type": "string" },
                            "output": { "type": "string" },
                },
            "required": ["explanation", "output"],
            "additionalProperties": False
            },
            "strict": True
        }
    }

Response structure

In [2]:
class QueryParams(BaseModel):
    query: str = Field(..., description="The SQL or search query to execute.")
    database: Optional[str] = Field(None, description="The target database name, if applicable.")
    limit: Optional[int] = Field(10, description="Maximum number of results to return.")

query_schema = QueryParams.schema()
query_schema

NameError: name 'BaseModel' is not defined

In [4]:
from pydantic import BaseModel, Field
from typing import Literal, Optional
from utils.function_to_schemas import function_to_schema


class GetTimeParams(BaseModel):
    location: str = Field(..., description="City and country e.g. Bogotá, Colombia")
    timezone: Optional[str] = Field(None, description="Timezone to convert the time to.")

get_time_schema = GetTimeParams.model_json_schema()
get_time_schema


{'properties': {'location': {'description': 'City and country e.g. Bogotá, Colombia',
   'title': 'Location',
   'type': 'string'},
  'timezone': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
   'default': None,
   'description': 'Timezone to convert the time to.',
   'title': 'Timezone'}},
 'required': ['location'],
 'title': 'GetTimeParams',
 'type': 'object'}

Function to Schema


In [6]:

tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Retrieves current weather for the given location.",
    "strict": True,
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            },
            "units": {
                "type": ["string", "null"],
                "enum": ["celsius", "fahrenheit"],
                "description": "Units the temperature will be returned in."
            }
        },
        "required": ["location", "units"],
        "additionalProperties": False
    }
},
{
    "type": "function",
    "name": "get_time",
    "description": "Retrieves current time for the given location.",
    "strict": True,
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            },
            "timezone": {
                "type": ["string", "null"],
                "description": "Timezone to convert the time to."
            }
        },
    "required": ["location", "timezone"],
    "additionalProperties": False
}
}
]

In [1]:
from openai import OpenAI
client = OpenAI()
from agent import Agent
import logging
from tools.get_weather import get_weather, WeatherParams
from utils.function_to_schemas import function_to_schema
logging.basicConfig(
    level=logging.INFO,  # or logging.DEBUG for more detail
    format='%(asctime)s - %(levelname)s - %(message)s'
)


In [2]:

system_prompt = "You are a helpful agent"
tools = {"get_weather": get_weather}
tool_schemas = [function_to_schema(get_weather, WeatherParams)]
tool_models = {"get_weather": WeatherParams}
agent = Agent("SalesAgent", client, system_prompt, tool_schemas, tool_models, tools)

In [ ]:
agent.response("What's the weather in Paris today?")


2025-04-19 17:05:16,363 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-04-19 17:05:17,172 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


'The weather in Paris today is sunny, with a temperature of 25°C.'

In [8]:
def load_prompt(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        return f.read()
prompt_template = load_prompt(r".\prompts\query_dataframe_tool\prompt_v1.txt")
prompt_template

'You are an expert data analyst.\n\n    You have access to a tool called `query_dataframe` that accepts a Python expression as input and runs it on a DataFrame called `df`.\n\n    Here is the DataFrame schema:\n    {schema_description}\n\n    Below are a few one-shot examples of valid filtering queries:\n\n    Example 1:\n    User Request: "List all leads from City \'Metropolis\' that have not been converted"\n    Expected tool call: query_dataframe("df[(df[\'City\'] == \'Metropolis\') & (df[\'Converted\'] == 0)]")\n\n    Example 2:\n    User Request: "Show me leads from Source \'Web\' with a lead score above 100"\n    Expected tool call: query_dataframe("df[(df[\'Lead Source\'] == \'Web\') & (df[\'Lead Score\'] > 100)]")\n\n    Example 3:\n    User Request: "Find leads that are qualified and in the age group \'30-35\'"\n    Expected tool call: query_dataframe("df[(df[\'Lead Stage\'] == \'Qualified\') & (df[\'Age\'] == \'30-35\')]")\n\n\n    Now, a user has asked to filter the DataFram

In [ ]:
schema_description ="HELLO WORLD"
user_request = "What is the weather in Paris today?"
prompt = prompt_template.format(
    schema_description=schema_description,
    user_request=user_request
)


You are an expert data analyst.

    You have access to a tool called `query_dataframe` that accepts a Python expression as input and runs it on a DataFrame called `df`.

    Here is the DataFrame schema:
    HELLO WORLD

    Below are a few one-shot examples of valid filtering queries:

    Example 1:
    User Request: "List all leads from City 'Metropolis' that have not been converted"
    Expected tool call: query_dataframe("df[(df['City'] == 'Metropolis') & (df['Converted'] == 0)]")

    Example 2:
    User Request: "Show me leads from Source 'Web' with a lead score above 100"
    Expected tool call: query_dataframe("df[(df['Lead Source'] == 'Web') & (df['Lead Score'] > 100)]")

    Example 3:
    User Request: "Find leads that are qualified and in the age group '30-35'"
    Expected tool call: query_dataframe("df[(df['Lead Stage'] == 'Qualified') & (df['Age'] == '30-35')]")


    Now, a user has asked to filter the DataFrame with this request:
    'What is the weather in Paris tod

In [4]:
agent.memory

[{'role': 'developer', 'content': 'You are a helpful agent'},
 {'role': 'user', 'content': "What's the weather in Paris today?"},
 {'type': 'function_call_output',
  'call_id': 'call_ACwMwh8MU7cJWcpgD16Bivli',
  'output': "Error: 'str' object has no attribute 'location'"}]

In [ ]:
agent.tools

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Retrieves current weather for the given location.',
  'strict': True,
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'City and country e.g. Bogotá, Colombia'},
    'units': {'type': ['string', 'null'],
     'enum': ['celsius', 'fahrenheit'],
     'description': 'Units the temperature will be returned in.'}},
   'required': ['location', 'units'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'get_time',
  'description': 'Retrieves current time for the given location.',
  'strict': True,
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'City and country e.g. Bogotá, Colombia'},
    'timezone': {'type': ['string', 'null'],
     'description': 'Timezone to convert the time to.'}},
   'required': ['location', 'timezone'],
   'additionalProperties': False}}]